In [1]:
""" Step1 引入需要的套件 """
import os
import pandas as pd
import numpy as np
import openpyxl
import random
from keras.layers import Dense
from keras.models import Sequential
import tensorflow as tf
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import regularizers
from tensorflow.keras.callbacks import EarlyStopping

""" 關閉 GPU 運行 (改用 CPU) """
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"  # 用GPU訓練無法固定隨機種子改用CPU訓練來行

""" Step3 建立 隨機種子碼 """
""" 種子碼設定 """
seed = 999
os.environ['PYTHONHASHSEED'] = str(seed)  # 固定 Python 產生的隨機性：在每次執行時一致
random.seed(seed)                         # 固定 random 套件的隨機性
np.random.seed(seed)                      # 固定 NumPy 的隨機性
tf.random.set_seed(seed)                  # 固定 TensorFlow 的隨機性

""" 建立 NN5 模型 """
M = Sequential([
    Dense(32, activation='relu', input_dim=1858,
          kernel_regularizer=regularizers.l1(1e-4)),   # 輸入層-第1層
    Dense(16, activation='relu'),                        # 第2層
    Dense(8, activation='relu'),                         # 第3層
    Dense(4, activation='relu'),                         # 第4層
    Dense(2, activation='relu'),                         # 第5層
    Dense(1)                          # 輸出層 (迴歸輸出)
])

""" learning rate為學習率 (速度) """
adam = Adam(learning_rate=0.01)

""" loss為損失函數，optimizer為優化器 """
M.compile(optimizer=adam, loss='mean_squared_error')
""" 根據驗證集停止訓練 """
early_stopping = EarlyStopping(
    monitor='val_loss',          # 監控的指標：驗證損失
    patience=5,                  # 容忍 5 個 epoch 沒有改善
    mode='min',                  # 指標愈小愈好（適用於 loss）
    restore_best_weights=True    # 停止回復到 val_loss 最佳的模型權重
)


C:\Users\a0900\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [2]:
""" Step4 預測 IC 值 """
""" 載入 Excel 檔案 """
workbook = openpyxl.load_workbook("NN5.xlsx")  # 如果檔案不存在，請先建立
sheet = workbook["預測IC"]                     # 選擇工作表

n0 = 48  # 預設最後回測 4 年 = 48 個觀察值（週）

""" 讀取 bm 的IC 值 """
df_x = pd.read_excel('bm.xlsx', sheet_name="bm補值")
df_y = pd.read_excel('bm.xlsx', sheet_name="bmIC")
df_x = df_x.T
df_y = df_y.T
df_x = df_x.iloc[2:, :]
df_y = df_y.iloc[2:, :]
n1, n2 = 178, 312

for n in range(n1, n2):
    X = df_x.iloc[0:n,:].values
    Y = df_y.iloc[1:n+1,:].values.ravel()
    test_X = df_x.iloc[n:n+1,:].values

    # M.fit X 需是 ndarray，故轉成 numpy float64
    X = X.astype('float64')
    Y = Y.astype('float64')
    test_X = test_X.astype('float64')

    """ 訓練集 """
    X_train = X[0:n-n0, :]
    Y_train = Y[0:n-n0]

    # 驗證集回測近4年 n0 = 48 = 12*4
    X_val = X[n-n0:n, :]
    Y_val = Y[n-n0:n]

    M.fit(X_train, Y_train,
          validation_data=(X_val, Y_val),
          epochs=100,                  # 最多訓練 100 個 epochs
          batch_size=32,              # 每批訓練資料大小為 32
          callbacks=[early_stopping]) # 使用 EarlyStopping 來防止過擬合

    predIC = M.predict(test_X)
    sheet.cell(row=3, column=n-n1+3, value=predIC[0,0])

""" 儲存檔案 """
workbook.save("NN5.xlsx")




Epoch 1/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 57ms/step - loss: 0.1673 - val_loss: 0.0837
Epoch 2/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - loss: 0.1195 - val_loss: 0.1081
Epoch 3/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - loss: 0.1347 - val_loss: 0.1067
Epoch 4/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - loss: 0.1325 - val_loss: 0.0978
Epoch 5/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - loss: 0.1216 - val_loss: 0.0821
Epoch 6/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - loss: 0.1064 - val_loss: 0.0680
Epoch 7/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - loss: 0.0932 - val_loss: 0.0564
Epoch 8/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - loss: 0.0819 - val_loss: 0.0474
Epoch 9/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - loss: 0.0734 - val_loss: 0.0400
Epoch 10/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - loss: 0.0663 - val_loss: 0.0345
Epoch 11/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - loss: 0.0608 - val_loss: 0.0299
Epoch 12/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - loss: 0.0563 - val_lo

In [3]:
df_x = pd.read_excel('size.xlsx', sheet_name="size補值")
df_y = pd.read_excel('size.xlsx', sheet_name="sizeIC")
df_x = df_x.T
df_y = df_y.T
df_x = df_x.iloc[2:, :]
df_y = df_y.iloc[2:, :]
n1, n2 = 178, 312
for n in range(n1, n2):
    X = df_x.iloc[0:n,:].values
    Y = df_y.iloc[1:n+1,:].values.ravel()
    test_X = df_x.iloc[n:n+1,:].values

    # M.fit X 需是 ndarray，故轉成 numpy float64
    X = X.astype('float64')
    Y = Y.astype('float64')
    test_X = test_X.astype('float64')

    """ 訓練集 """
    X_train = X[0:n-n0, :]
    Y_train = Y[0:n-n0]

    # 驗證集回測近4年 n0 = 48 = 12*4
    X_val = X[n-n0:n, :]
    Y_val = Y[n-n0:n]

    M.fit(X_train, Y_train,
          validation_data=(X_val, Y_val),
          epochs=100,                  # 最多訓練 100 個 epochs
          batch_size=32,              # 每批訓練資料大小為 32
          callbacks=[early_stopping]) # 使用 EarlyStopping 來防止過擬合

    predIC = M.predict(test_X)
    sheet.cell(row=4, column=n-n1+3, value=predIC[0,0])

""" 儲存檔案 """
workbook.save("NN5.xlsx")

Epoch 1/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - loss: 0.0274 - val_loss: 0.0183
Epoch 2/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - loss: 0.0235 - val_loss: 0.0168
Epoch 3/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - loss: 0.0229 - val_loss: 0.0182
Epoch 4/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - loss: 0.0248 - val_loss: 0.0185
Epoch 5/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - loss: 0.0248 - val_loss: 0.0162
Epoch 6/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - loss: 0.0224 - val_loss: 0.0157
Epoch 7/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - loss: 0.0219 - val_loss: 0.0160
Epoch 8/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - loss: 0.0222 - val_loss: 0.0159
Epoch 9/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - loss: 0.0222 - val_loss: 0.0165
Epoch 10/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - loss: 0.0226 - val_loss: 0.0165
Epoch 11/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - loss: 0.0229 - val_loss: 0.0156
Epoch 12/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - loss: 0.0222 - val_lo

In [4]:
""" 預測 mom 的 IC """
df_x = pd.read_excel('mom.xlsx', sheet_name="mom補值")
df_y = pd.read_excel('mom.xlsx', sheet_name="momIC")
df_x = df_x.T
df_y = df_y.T
df_x = df_x.iloc[2:, :]
df_y = df_y.iloc[2:, :]
n1, n2 = 166, 300
for n in range(n1, n2):
    X = df_x.iloc[0:n, :].values
    Y = df_y.iloc[1:n+1, :].values.ravel()
    test_X = df_x.iloc[n:n+1, :].values
    X = X.astype('float64')
    Y = Y.astype('float64')
    test_X = test_X.astype('float64')

    # 訓練集
    X_train = X[0:n-n0, :]
    Y_train = Y[0:n-n0]
    
    # 驗證集回測近4年 n0 = 48 = 12*4
    X_val = X[n-n0:n, :]
    Y_val = Y[n-n0:n]

    M.fit(X_train, Y_train,
          validation_data=(X_val, Y_val),
          epochs=100,
          batch_size=32,
          callbacks=[early_stopping])
predIC = M.predict(test_X)
sheet.cell(row=5, column=n-n1+3, value=predIC[0,0])

""" 儲檔 """
workbook.save("NN5.xlsx")




Epoch 1/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step - loss: 0.0296 - val_loss: 0.0218
Epoch 2/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - loss: 0.0299 - val_loss: 0.0210
Epoch 3/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - loss: 0.0301 - val_loss: 0.0207
Epoch 4/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - loss: 0.0303 - val_loss: 0.0216
Epoch 5/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - loss: 0.0307 - val_loss: 0.0213
Epoch 6/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - loss: 0.0307 - val_loss: 0.0212
Epoch 7/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - loss: 0.0297 - val_loss: 0.0209
Epoch 8/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - loss: 0.0294 - val_loss: 0.0206
Epoch 9/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - loss: 0.0293 - val_loss: 0.0207
Epoch 10/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - loss: 0.0295 - val_loss: 0.0208
Epoch 11/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - loss: 0.0299 - val_loss: 0.0209
Epoch 12/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - loss: 0.0301 - val_lo

In [5]:
""" Step5 載入數據 """
IC = pd.read_excel("NN5.xlsx", sheet_name="預測IC").T
IC = np.array(IC.iloc[2:,1:4])
# 載入 bm（基準因子）資料
bm = pd.read_excel('bm.xlsx', sheet_name="bm補值").T
bm = np.array(bm.iloc[181:,:])
bm_ret = pd.read_excel('bm.xlsx', sheet_name="下個月月報酬補值").T
bm_ret = np.array(bm_ret.iloc[181:,:])

# 載入 size（市值因子）資料
size = pd.read_excel('size.xlsx', sheet_name="size補值").T
size = np.array(size.iloc[181:,:])
size_ret = pd.read_excel('size.xlsx', sheet_name="下個月月報酬補值").T
size_ret = np.array(size_ret.iloc[181:,:])

# 載入 mom（動能因子）資料
mom = pd.read_excel('mom.xlsx', sheet_name="mom補值").T
mom = np.array(mom.iloc[169:,:])
mom_ret = pd.read_excel('mom.xlsx', sheet_name="下個月月報酬補值").T
mom_ret = np.array(mom_ret.iloc[169:,:])


In [6]:
""" Step6 計算投資組合 """
# 2013/12~2025/01 共 134 個月
for n in range(134):
    c = pd.DataFrame()
    sel_index = np.argmax(abs(IC[n]))        # 找出當月絕對值最大的 IC
    sel_value = IC[n][sel_index]

    # 選擇因子與報酬
    if sel_index == 0:
        fac, sel, ret = bm[n], 'bm', bm_ret[n]
    elif sel_index == 1:
        fac, sel, ret = size[n], 'size', size_ret[n]
    elif sel_index == 2:
        fac, sel, ret = mom[n], 'mom', mom_ret[n]

    c['factor'] = fac
    c['return'] = ret
    c['rank'] = c['factor'].rank(method='max')
    c = c.sort_values(by='rank')

    row = [11,12,13,14,15,16,17,18,19,20,21,22,23,24]
    group = [185,92,37,18,10,9,8,7,6,5,4,3,2,1]

    for r, g in zip(row, group):
        high = np.mean(c['return'].iloc[-g:])
        low = np.mean(c['return'].iloc[:g])
        ans = high - low if sel_value > 0 else low - high
        sheet.cell(row=r, column=n+3, value=ans)

""" 儲檔 """
workbook.save("NN5.xlsx")
